In [3]:
import re
import pandas as pd
import numpy as np

from brainglobe_registration.elastix.register import (
    run_registration
)
from brainglobe_registration.utils.utils import open_parameter_file
from pathlib import Path


In [4]:
import cv2

def load_video_to_numpy(video_path):
    cap = cv2.VideoCapture(video_path)
    frames = []
    
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        frames.append(frame.astype(np.uint8))
    
    cap.release()
    return np.array(frames, dtype=np.uint8)

video_path = '../21Jan_007.mp4'
video_data = load_video_to_numpy(video_path)
print(video_data.shape)

(6294, 1080, 1920)


In [3]:
video_data.dtype

dtype('uint8')

In [5]:
file_path = (
    Path.home() / "NIU-dev" / "brainglobe-registration"
    / "brainglobe_registration"
    / "parameters"
    / "brainglobe_registration"
    / "affine.txt"
)

parameter_lists = [("affine", open_parameter_file(file_path))]
parameter_lists[0][1]["FixedInternalImagePixelType"] = ["float"]
parameter_lists[0][1]["MovingInternalImagePixelType"] = ["float"]
parameter_lists[0][1]["NumberOfResolutions"] = ["2"]
parameter_lists[0][1]["MaximumNumberOfIterations"] = ["200"]
parameter_lists[0][1]["Transform"] = ["EulerTransform"]

In [6]:
from tqdm.notebook import tqdm

output_file = "out_euler_frame.csv"

with open(output_file, "w") as f:
    f.write("theta,tx,ty\n")

for i in tqdm(range(1, video_data.shape[0])):
    fixed_gray = video_data[i-1]
    moving_gray = video_data[i]
    parameters = run_registration(
        fixed_gray,
        moving_gray,
        parameter_lists,
        filter_images=False
    )

    # Regular expression to find the TransformParameters line
    pattern = r'\(TransformParameters ([\d\.\-e ]+)\)'
    input_string = str(parameters)
    # Search for the pattern in the input string
    match = re.search(pattern, input_string)

    if match:
        # Extract the numbers and convert them to floats
        transform_parameters = list(map(float, match.group(1).split()))
        with open(output_file, "a") as f:
            f.write(",".join(map(str, transform_parameters)))
            f.write("\n")
    else:
        print("TransformParameters not found")

  0%|          | 0/6293 [00:00<?, ?it/s]

In [8]:
output_file = "out_euler_frame.csv"

df = pd.read_csv(output_file)
## Add the first frame row
df = pd.concat([pd.DataFrame({
    'theta': [0],
    'tx': [0],
    'ty': [0]
}), df], ignore_index=True)
df.head()

,theta,tx,ty
0,0.000000,0.00000,0.000000
1,0.000037,1.06339,0.450534
2,0.000009,1.19702,0.424645
3,0.000083,1.09340,0.409683
4,0.000071,1.12484,0.444346


In [9]:
df['tx_sum'] = df['tx'].cumsum()
df['ty_sum'] = df['ty'].cumsum()

In [10]:
df['tx_sum_int'] = df['tx_sum'].round(0).astype(int)
df['ty_sum_int'] = df['ty_sum'].round(0).astype(int)

x_min = df['tx_sum_int'].min()
x_max = df['tx_sum_int'].max()
y_min = df['ty_sum_int'].min()
y_max = df['ty_sum_int'].max()
print(x_min, x_max, y_min, y_max)

0 3865 -273 597


In [11]:
height, width = video_data.shape[1:3]

total_height = y_max - y_min + height
total_width = x_max - x_min + width

print(total_height, total_width)

1950 5785


In [13]:
import numpy as np
fused_image = np.zeros((total_height, total_width), dtype=np.uint8)

for i in range(video_data.shape[0] - 1, 0, -1):
    x = df['tx_sum_int'][i] - x_min
    y = df['ty_sum_int'][i] - y_min
    fused_image[y:y+height, x:x+width] = video_data[i]

In [14]:
import tifffile

tifffile.imwrite("fused_image.tif", fused_image)